## Notebook to fetch and compile stonycreek into one excel file

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from datetime import datetime
from plotly.subplots import make_subplots
import requests
import io
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interact
import numpy as np
from IPython.display import display
import re

pd.options.plotting.backend = "plotly"

In [ ]:
SITE_PREFIX = "Stony"
SITE_NUMBERS = [f"0{x}" for x in (1,2,3,4,6,7)] + ["05-1", "05-3"]
SITE_NAMES = [f"{SITE_PREFIX}{x}" for x in SITE_NUMBERS]
print(SITE_NAMES)
BASE_URL = "https://www.bosl.com.au/IoT/StonyCreek/databases/"

In [ ]:
all_data_v1 = []
all_data_v2 = []
for site in SITE_NAMES:
    print(f"Hit: {BASE_URL}{site}.csv, {BASE_URL}{site}_v2.csv")
    all_data_v1.append(pd.read_csv(f"{BASE_URL}{site}.csv").dropna(how='all', axis='columns'))
    all_data_v2.append(pd.read_csv(f"{BASE_URL}{site}_v2.csv").dropna(how='all', axis='columns'))

    # Misc data cleaning
    # 05-1 has extra row
    #if site == f"{SITE_PREFIX}05-1":
    #    all_data_v1[-1].drop(all_data_v1[-1].tail(1).index, inplace=True)

In [ ]:
for data_v1, data_v2 in zip(all_data_v1, all_data_v2):
    COLUMN_CONVERSION = {"DateTime": "SiteName", "Battery": "Velocitybattery", "Velocity": "VEL", "ApproxDepth": "DIST", "SignalStrength": "AMP",
        "BinaryDepth": "WPEAKS", "Angle": "ANGLE"}
    data_v1.rename(columns=COLUMN_CONVERSION, inplace=True)
    data_v2.rename(columns={"PEAKS": "WPEAKS"}, inplace=True)

In [ ]:
all_merged_data = []
for data_v1, data_v2 in zip(all_data_v1, all_data_v2):
    merged_data = pd.concat([data_v1, data_v2])
    merged_data.dropna(how='all', subset=merged_data.columns, inplace=True)
    merged_data.dropna(axis=1, inplace=True, how='all')
    merged_data.dropna(subset=["WPEAKS"], inplace=True)
    try:
        def extract_peak(row):
            peaks = str(row["WPEAKS"]).split("|")
            try:
                decoded_peaks = [int(x[:3], 16) for x in peaks]
            except ValueError:
                decoded_peaks = [None] * len(peaks)
            finally:
                column_names = [f"DEPTH-{i+1}" for i in range(len(decoded_peaks))]
                return pd.Series(decoded_peaks, index=column_names)
        merged_data = pd.concat([merged_data, merged_data.apply(extract_peak, axis=1)], axis=1)
        depth_column_names = [col for col in merged_data.columns if 'DEPTH' in col]
        
        merged_data[depth_column_names] = merged_data[depth_column_names].multiply(np.sin(np.radians(merged_data["ANGLE"])), axis="index")
        merged_data["SiteName"] = pd.to_datetime(merged_data["SiteName"], format="%d/%m/%y %I:%M:%S %p")

        merged_data.dropna(subset=depth_column_names, axis='rows', inplace=True)

        all_merged_data.append(merged_data)
        merged_data.dropna(how='all', axis=1, inplace=True)
    except ValueError as e:
        print(merged_data)
        raise e
merged_data.dropna(how='all', axis=1, inplace=True)

Negative depths are removed

In [ ]:
merged_data.loc[merged_data["DEPTH-1"] < 0]

In [ ]:
for merged_data in all_merged_data:
    merged_data.drop(merged_data[merged_data["DEPTH-1"] < 0].index, inplace=True)

In [ ]:
fig = px.line(all_merged_data[0], x='SiteName', y=depth_column_names)
f2 = go.FigureWidget(fig)
site_id = widgets.IntSlider(value=0, min=0, max=len(SITE_NAMES) - 1, step=1, description='Site index:', continuous_update=False)
def response(change):
    f2.update_layout(title=f"Depth data of {SITE_NAMES[site_id.value]}")
    with f2.batch_update():
        for idx, col in enumerate(depth_column_names):
            f2.data[idx].x = all_merged_data[site_id.value]["SiteName"]
            f2.data[idx].y = all_merged_data[site_id.value][col]
site_id.observe(response, names="value")
response(None)
widgets.VBox([site_id, f2])

In [ ]:
earliest_date = np.min(list(x["SiteName"].min() for x in all_merged_data)).date()
latest_date = np.max(list(x["SiteName"].max() for x in all_merged_data)).date()

url = f"https://api.melbournewater.com.au/rainfall-river-level/227231A/rain/hourly/range.csv?fromDate={earliest_date}&toDate={latest_date}"
print(url)
r = requests.get(url)
d = r.headers['content-disposition']
rain_filename = re.findall("filename=(.+)", d)[0]
rainfall_data = pd.read_csv(io.StringIO(r.content.decode('utf-8'))).dropna(how='all', axis='columns')

In [ ]:
rainfall_data["Date/Time"] = pd.to_datetime(rainfall_data["Date/Time"], format="%Y/%m/%d %H:%M:%S")
rainfall_data

In [ ]:
fig = rainfall_data.plot(x='Date/Time', y="Current rainfall (mm)", title=f"Rainfall of {rain_filename}").update_layout(
    xaxis_title="Time", yaxis_title=f"Rainfall (mm)"
)
fig.show()

Periods in which there is a >1 hr gap is ignored

In [ ]:
time_filter = lambda x: x > 1 * 60 * 60

In [ ]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=merged_data["SiteName"],
#     y=np.where(time_filter(merged_data['SiteName'].diff().dt.seconds), None, merged_data["DEPTH-1"]), 
#     name="Depth"),
#     secondary_y=False)
# fig.add_trace(go.Scatter(x=rainfall_data["Date/Time"], y=rainfall_data["Current rainfall (mm)"], name="Rainfall"), secondary_y=True)
# fig.update_yaxes(title_text="Depth (mm)", secondary_y=False)
# fig.update_yaxes(autorange="reversed", secondary_y=True)
# fig.update_yaxes(title_text="Rainfall (mm)", secondary_y=True)
# fig.update_xaxes(title_text="Time", tickangle=45, nticks=30)
# fig.update_layout(autosize=True, width=1100, height=700, legend=dict(title="Lines"), title=f"Rainfall vs Depth of {site}")
# fig.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for site_name, merged_data in zip(SITE_NAMES, all_merged_data):
    fig.add_trace(go.Scatter(x=merged_data["SiteName"],
    y=np.where(time_filter(merged_data['SiteName'].diff().dt.seconds), None, merged_data["DEPTH-1"]), 
    name=f"{site_name}"),
    secondary_y=False)
    
fig.add_trace(go.Scatter(x=rainfall_data["Date/Time"], y=rainfall_data["Current rainfall (mm)"], name="Rainfall"), secondary_y=True)
fig.update_yaxes(title_text="Depth (mm)", secondary_y=False, range = [0, 8000])
max_rainfall = rainfall_data["Current rainfall (mm)"].max()
fig.update_yaxes(range = [max_rainfall * 4, 0], secondary_y=True)
fig.update_yaxes(title_text="Rainfall (mm)", secondary_y=True)
fig.update_xaxes(title_text="Time", tickangle=45, nticks=30)
fig.update_layout(autosize=False, width=1100, height=700, legend=dict(title="Lines"), title="All plots")

fig.show()

In [ ]:
writer = pd.ExcelWriter(f'stony_creek-{datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")}.xlsx', engine='xlsxwriter')
for site_name, merged_data in zip(SITE_NAMES, all_merged_data):
    merged_data.to_excel(writer, sheet_name=site_name, index=False)
rainfall_data.to_excel(writer, sheet_name=f"{rain_filename[:31]}", index=False)
writer.close()